In [10]:
import pandas as pd
import json as js
import matplotlib.pyplot as plt
%matplotlib inline

#### Retrieve Data

In [11]:
raw_data_movies = pd.read_csv('movies.csv', sep=',')
raw_data_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
raw_data_ratings = pd.read_csv('ratings.csv', sep=',')
raw_data_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


#### Clear Data

1/ Normalize data : each movie sums its ratings at 1

In [13]:
all_movies_id = raw_data_ratings.movieId.unique()

for movie_id in all_movies_id:
    total_rating = sum(raw_data_ratings[raw_data_ratings.movieId==movie_id].rating)
    raw_data_ratings.loc[raw_data_ratings.movieId==movie_id,'rating'] = raw_data_ratings.loc[raw_data_ratings.movieId==movie_id,'rating']/total_rating

2/ Binarize data : ratings > mean(others movies for one user) => 1 

else => 0

In [14]:
all_users_id = raw_data_ratings.userId.unique()

for user_id in all_users_id:
    temp_ratings = raw_data_ratings.loc[raw_data_ratings.userId == user_id].rating.copy()
    i = temp_ratings.index[0]
    mean_temp_ratings = temp_ratings.mean()
    
    for curt_rate in temp_ratings:
        if curt_rate>mean_temp_ratings:
            temp_ratings[i] = 1
        else:
            temp_ratings.loc[i,] = 0
        i+= 1

    raw_data_ratings.loc[raw_data_ratings.userId==user_id,'rating'] = temp_ratings

3/ Remove users with less than 20 movies rated 1

Remove movies with less than 10 rates to 1

In [15]:
def clear_users(data):
    all_users_id = data.userId.unique()
    nbr_clear = 0
    for user_id in all_users_id:
        temp_ratings = data.loc[data.userId == user_id].rating.copy()
        nbr_ratings = sum(temp_ratings==1)    
        if nbr_ratings < 20: 
            data = data[data.userId != user_id]
            nbr_clear += 1
    return data, nbr_clear

def clear_movies(data):
    all_movies_id = data.movieId.unique()
    nbr_clear = 0
    for movie_id in all_movies_id:
        temp_ratings = data.loc[data.movieId==movie_id].rating.copy()
        nbr_ratings = sum(temp_ratings==1)    
        if nbr_ratings < 10:
            data = data[data.movieId != movie_id]
            nbr_clear += 1
    return data, nbr_clear 

In [16]:
nbr_clear = 1
while nbr_clear > 0:
    raw_data_ratings, nbr_clear = clear_movies(raw_data_ratings)
    print('movies clear : ', nbr_clear)
    raw_data_ratings, nbr_clear = clear_users(raw_data_ratings)
    print('users clear : ', nbr_clear)

movies clear :  8976
users clear :  480
movies clear :  48
users clear :  0


In [17]:
raw_data_ratings.shape

(0, 4)